In [ ]:
# Only need to do this once...
!pip install flask
!pip install --upgrade google-cloud-datastore
!pip install Flask-Mail
!pip install scikit-surprise

In [ ]:

import json
import re
from flask import Flask, request, jsonify, make_response
from flask import render_template
from google.cloud import datastore

import os
from surprise import SVD
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import BaselineOnly
from surprise import Reader
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
import pandas as pd

In [ ]:
from send_invitation import send_email, send_email_request
from threading import Thread
import jwt

app = Flask(__name__)
#Quantity of the recommond results
top_k = 5
recResults = dict()

import re
from random import seed
from random import randint
from datetime import datetime

seed(datetime.now())

app.config['SECRET_KEY'] = 'SECRET_KEY_CHAT9900'

file_path = os.path.expanduser('ratings_small.csv')
movie_path = "movies_metadata.csv"
# friend_path = "friends.csv"

reader = Reader(line_format='user item rating timestamp', sep=',',skip_lines=1)
surprise_data = Dataset.load_from_file(file_path, reader=reader)
movies_df = pd.read_csv(movie_path)

#chatroom public url
def get_chatroom_url():
    # populate this from your local server
    return "http://c801f39d04b7.ngrok.io"

# url of the webhook. 
# it is done by curl the local ngrok api
# It is for friend request validations [extra feature]
def get_public_url():
    # the https string is from ngrok
    # https://gist.github.com/rjz/af40158c529d7c407420fc0de490758b
    
    tunnel = !curl http://127.0.0.1:4040/api/tunnels
    tunnel= str(tunnel)
    print(tunnel)
    mobj = re.search('public_url', tunnel)
    i = mobj.span()
    t = tunnel[i[1]:].split('\"')
    public_url =t[2]
    return public_url

def gen_jwt(seedstr):
    return jwt.encode(
            {'requester': seedstr },
            app.config['SECRET_KEY'], algorithm='HS256')

def gen_jwt_request(requester, receiver):
    return jwt.encode(
            {
                'requester': requester,
                'receiver': receiver,
            },
            app.config['SECRET_KEY'], algorithm='HS256')

def get_user_from_email(email):
    client = datastore.Client()
    query = client.query(kind="users")
    query.add_filter("email", "=", email)
    result = list(query.fetch(1))
    if len(result) == 0:
        return {
          "fulfillmentText": 
                  "logged in user email not found. Please contact admin chatbotchatbot9900@gmail.com"
        }, 0
    return result, 1

def get_email_from_id(uid):
    client = datastore.Client()
    query = client.query(kind="users")
    query.add_filter("id", "=", uid)
    result = list(query.fetch(1))
    print("get_email_from_id", result, uid, type(uid))
    if len(result) == 0:
        return {
          "fulfillmentText": 
                  "logged in user id not found. Please contact admin chatbotchatbot9900@gmail.com"
        }, 0
    return result, 1

def get_friends_from_id(fid):
    client = datastore.Client()
    query = client.query(kind="friends")
    query.add_filter("id", "=", fid)
    result = list(query.fetch(1))
    if len(result) == 0:
        return {
          "fulfillmentText": 
                  "friend id not found. Please contact admin chatbotchatbot9900@gmail.com"
        }, 0
    return result, 1

# [extra feature]
@app.route('/friend/<token>', methods=['GET', 'POST'])
def validate_friend_requests(token):
    requester = jwt.decode(token, app.config['SECRET_KEY'],
                            algorithms=['HS256'])['requester']
    receiver = jwt.decode(token, app.config['SECRET_KEY'],
                            algorithms=['HS256'])['receiver']
    print("requester receiver",requester, receiver)
    u1, r1 = get_user_from_email(requester)
    u2, r2 = get_user_from_email(receiver)
    f1, r3 = get_friends_from_id(u1[0].get('id'))
    f2, r4 = get_friends_from_id(u2[0].get('id'))
    if (r1 == 0 or r2 == 0 or r3 == 0 or r4 == 0 ):
        render_template('error.html')
    print(u1)
    print(u2)
    print(f1)
    print(f2)
    fids1 = set(f1[0].get("fids"))
    fids2 = set(f2[0].get("fids"))
    #print(fids1, type(fids1))
    fids1.add(int(u2[0].get('id').decode("utf-8")))
    fids2.add(int(u1[0].get('id').decode("utf-8")))
    #print(fids1)
    datastore_client = datastore.Client()
    with datastore_client.transaction():
        f1[0].update(
            {
                "id": f1[0].get("id"),
                "fids": list(fids1),
            }
        )
        datastore_client.put(f1[0])
    with datastore_client.transaction():
        f2[0].update(
            {
                "id": f2[0].get("id"),
                "fids": list(fids2),
            }
        )
        datastore_client.put(f2[0])
    print(f1, f2)
    return render_template('accept.html')

@app.route('/webhook/', methods=['POST'])
def handle():
    req = request.get_json(silent=True, force=True)
    print('Request:')
    print(json.dumps(req, indent=4))
    intent = req.get('queryResult').get('intent').get('displayName')
    print(intent)
    # there could be multiple topics in a sentence
    if isinstance(intent, list):
        intent = intent[0]
    #topic = re.sub(r'[^\w\s]', '', topic)
    
    rsp = buildReply('Command not recognized')
    if (intent == "Schedule: Movie - yes"):
        rsp = scheduleMovie(req)
    elif (intent == "profile" or intent == "account"):
        rsp = getProfileAccount()
    elif (intent == "Recommendation:" ):
        rsp = getRecommendations(req)
    elif (intent == "RecFriends" ):
        rsp = getFriends(req)
    elif (intent == "login-prompt"):
        rsp = validateLogin(req)
    elif (intent == "friend request"):
        rsp = friendRequest(req)
    elif (intent=="Show my Stats"):
        rsp = displayStats(req)
        
    rsp = json.dumps(rsp, indent=4)
    print (rsp)
    r = make_response(rsp)
    r.headers['Content-Type'] = 'application/json'
    return r    
  
def displayStats(req):
    current_user_id = req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user_id')
    if current_user_id==None:
        return {
          "fulfillmentText": 
                  "Please log in first. (Type 'login')"
        }
    url = "https://cedrus422.wixsite.com/website/user-analytics-" + current_user_id
    return {
          "fulfillmentText": 
                  url
        }
# get similar users
def getSimilarUsers(top_k,u_id):
    all_trainset = surprise_data.build_full_trainset()
    algo = KNNBasic(k=40,min_k=3,sim_options={'user_based': True}) # sim_options={'name': 'cosine','user_based': True} cosine/msd/pearson/pearson_baseline
    algo.fit(all_trainset)
    user_inner_id = algo.trainset.to_inner_uid(u_id)
    user_neighbors = algo.get_neighbors(user_inner_id, k=top_k)
    user_neighbors = (algo.trainset.to_raw_uid(inner_id) for inner_id in user_neighbors)
    return user_neighbors

# get similar movies
def getSimilarItems(top_k,item_id):
    all_trainset = surprise_data.build_full_trainset()
    item_algo = KNNBasic(k=40,min_k=3,sim_options={'user_based': False}) # sim_options={'name': 'cosine','user_based': True} cosine/msd/pearson/pearson_baseline
    item_algo.fit(all_trainset)
    item_inner_id = item_algo.trainset.to_inner_iid(item_id)
    item_neighbors = item_algo.get_neighbors(item_inner_id, k=top_k)
    f_item_neighbors = (item_algo.trainset.to_raw_iid(inner_id)
                       for inner_id in item_neighbors)
    return f_item_neighbors

def getMoviesFromList(movieList):
    movies = []
    text = ""
    for movie in movieList:
        #print(movies)
        for i in range(len(movies_df)):
            if movies_df['id'][i] == movie:
                movies.append(movies_df['title'][i])
                if movie not in movies_df['id']:
                    movies.append("Movie ID"+movies_df['id'][i]+"(currently unavailable)")
                
    for m in movies:
        text += str(m)+"\n"
    print(text)
    return text

def getMoviesFromTitle(title):
    #get movie ID or report error
    movies = []
    for i in range(len(movies_df)):
        if str(movies_df['title'][i]) == str(title):
            moviesList = list(getSimilarItems(top_k,movies_df['id'][i]))
            break
        elif i==len(movies_df)-1:
            return print("Couldn't find the movie, Please try again.")
    print(moviesList)
    return getMoviesFromList(moviesList)
    
def getMoviesFromUser(userID):
    user_df = pd.read_csv('ratings_small.csv')
    maxRate = 0
    for i in range(len(user_df)):
        if (str(user_df['userId'][i]) == str(userID)) and (float(user_df['rating'][i]) > maxRate):
            maxRate = user_df['rating'][i]
            likedID = user_df['movieId'][i]

    recList = list(getSimilarItems(top_k,str(likedID)))
    return getMoviesFromList(recList)


def get_attendees(current_user_id, current_user_email, tofriend):
    attendees = list()
    attendees.append("chatbot9900@gmail.com")
    attendees.append(current_user_email)
    client = datastore.Client()
    query = client.query(kind="friends")
    query.add_filter("id", "=", current_user_id)
    result = list(query.fetch(1))
    print(current_user_id, result)
    print('fids',result[0].get('fids'))
    friends = result[0].get('fids')
    for f in friends:
        query = client.query(kind="users")
        query.add_filter("id", "=", str(f))
        query.add_filter("name", "=", tofriend)
        print(query)
        print(f, tofriend)
        result = list(query.fetch(1))
        if len(result) == 1 :
            print('email',result[0].get('email'))
            attendees.append(result[0].get('email').decode("utf-8"))
    return attendees
        
def scheduleMovie(req):
    tofriend = req.get('queryResult').get('outputContexts')[0].get('parameters').get('friend')[0].get('name')
    print(tofriend,'tofriend')
    movie = req.get('queryResult').get('outputContexts')[0].get('parameters').get('movie')[0]
    date = req.get('queryResult').get('outputContexts')[0].get('parameters').get('date')[0]
    time = req.get('queryResult').get('outputContexts')[0].get('parameters').get('time')[0]
    current_user_email = req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user')
    #
    print(current_user_email, tofriend)
    if (current_user_email == None):
        return {
          "fulfillmentText": 
                  "Please log in first. (Type 'login')"
        }
    
    client = datastore.Client()
    query = client.query(kind="users")
    query.add_filter("email", "=", current_user_email)
    result = list(query.fetch(1))
    if len(result) == 0:
        return {
          "fulfillmentText": 
                  "logged in user email not found. Please contact admin chatbotchatbot9900@gmail.com"
        }
    #print(result)
    #print(result[0])
    #print(result[0].id)
    #print(result[0].get('email'))
    #print(attendees)
    attendees = get_attendees(result[0].get("id"), current_user_email, tofriend)
    if (len(attendees) == 2):
        return buildReply('Target user email not found.')
    #print(str(datetime.now()))
    print(attendees)
    
    # token in this context is just a random string acting as chatroom id
    token = gen_jwt(str(datetime.now()))
    chatroom_url = get_chatroom_url()+"/"+token.decode("utf-8")
    Thread(target=send_email, args=(attendees,chatroom_url, movie, date.split('T')[0], time.split('T')[1]),).start()

    return buildReply('Awesome! Your movie meeting is confirmed. Sending request...')
  
def getProfileAccount():
    urls = ""
    with open('netflix_urls.txt') as netflix_urls:
        urls = netflix_urls.read()
    print (urls)
    return buildReply(urls)

def getRecommendations(req):
    global recResults
    current_user_id = req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user_id')
    if current_user_id==None:
        return {
          "fulfillmentText": 
                  "Please log in first. (Type 'login')"
        }
    return buildReply(str(recResults[current_user_id]))
    
def getFriends(req):
    global recFriends
    current_user_id = req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user_id')
    top_k = 5
    text = "User ID:\n"
    if current_user_id==None:
        return {
          "fulfillmentText": 
                  "Please log in first. (Type 'login')"
        }
    flist = getSimilarUsers(top_k,current_user_id)
    for f in flist:
        text += str(f)
        user, ret = get_email_from_id(str(f))
        if ret == 1:
            email = user[0].get('email').decode("utf-8")
            text += " "+ email
        text += "\n"
    return buildReply(text)
    
def getRecResults(uid):
    global recResults
    #get user id
    recResults[uid] = getMoviesFromUser(uid)
    #get rec list stored
    return
    
def validateLogin(req):
    email = req.get('queryResult').get('parameters').get("email")
    password = req.get('queryResult').get('parameters').get("password")
    session = req.get('queryResult').get('outputContexts')[0].get('name')
    client = datastore.Client()
    # you may have to create this kind manually in gcp datastore 
    query = client.query(kind="users")
    print(email,password)
    # These two columns has to be indexed in gcp datastore 
    # otherwise they can not be found and filtered
    query.add_filter("email", "=", email)
    query.add_filter("randomnumber", "=", password)
    result = list(query.fetch(1))    
    print("validateLogin",result)
    if len(result) == 1:
        uid = result[0].get("id").decode("utf-8")
        #New thread for recommendation
        Thread(target=getRecResults,args=(uid)).start()
        return confirmLogin(uid, email, session)
    return buildReply("Invalid combination")
  
def confirmLogin(id, un, sn):
    array = sn.split('/')
    session = ""
    for i in range(len(array)-1):
        session += array[i]+'/'
    session += "logged-in"
    return {
      "fulfillmentText": 
              "User: "+un+" logged in"
      ,
      "outputContexts": [
        {
          "name": session,
          "lifespanCount": 500,
          "parameters": {
            "loggedin_user": un,
            "loggedin_user_id": id
          }
        }
      ]
    }

def friendRequest(req):
    current_user_id = req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user_id')
    current_user_email = req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user')
    toemail = req.get('queryResult').get('outputContexts')[0].get('parameters').get('email')
    if (current_user_id == None):
        return {
          "fulfillmentText": 
                  "Please log in first. (Type 'login')"
        }
    request_link = get_public_url()+"/friend/"+gen_jwt_request(current_user_email, toemail).decode("utf-8")
    print(request_link)
    user, ret = get_user_from_email(toemail)
    if (ret == 0):
        return buildReply("Target user email not found in the datastore.")
    
    Thread(target=send_email_request, args=(current_user_email, toemail, request_link,)).start()
    return buildReply("Friend request sent")

def buildReply(info):
    return {
        'fulfillmentText': info,
    }

if __name__ == '__main__':
    app.run(host='0.0.0.0')